# Explore Inference Transcriptions

This notebook lets you:
1. Browse the completed inference results (144 samples)
2. Listen to the original audio
3. Compare original text vs model transcription
4. Compute WER/CER metrics
5. Try the teacher CoT prompt (Gemini-based analysis)

In [1]:
import json
import io
import os
import wave
import struct

import numpy as np
import tensorflow as tf
import grain.python as grain
from jiwer import wer, cer
from IPython.display import Audio, display, HTML, Markdown
import ipywidgets as widgets

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # suppress TF warnings

GCSFUSE_BASE = "/tmp/gcsfuse/grain_data_arrayrecord"
RESULTS_FILE = "/tmp/gcsfuse/distillation/inference_results.jsonl"
SAMPLE_RATE = 16000

2026-02-13 13:18:23.853822: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-13 13:18:23.854312: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-13 13:18:23.903820: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Load Inference Results

In [2]:
samples = []
with open(RESULTS_FILE, "r") as f:
    for line in f:
        line = line.strip()
        if line:
            samples.append(json.loads(line))

print(f"Loaded {len(samples)} transcription results")
print(f"\nFields: {list(samples[0].keys())}")
print(f"\nDatasets represented:")
from collections import Counter
ds_counts = Counter(s["dataset_name"] for s in samples)
for ds, count in ds_counts.most_common():
    print(f"  {ds}: {count}")

Loaded 144 transcription results

Fields: ['sample_id', 'dataset_name', 'audio_file', 'record_idx', 'original_text', 'model_transcription', 'duration']

Datasets represented:
  MohamedRashad/SADA22: 144


## 2. Basic Statistics

In [3]:
durations = [s["duration"] for s in samples]
print(f"Duration stats:")
print(f"  Min:    {min(durations):.2f}s")
print(f"  Max:    {max(durations):.2f}s")
print(f"  Mean:   {np.mean(durations):.2f}s")
print(f"  Median: {np.median(durations):.2f}s")
print(f"  Total:  {sum(durations)/60:.1f} min")

# Compute WER and CER for all samples
wer_scores = []
cer_scores = []
for s in samples:
    ref = s["original_text"].strip()
    hyp = s["model_transcription"].strip()
    if ref:  # skip empty references
        try:
            wer_scores.append(wer(ref, hyp))
            cer_scores.append(cer(ref, hyp))
        except Exception:
            pass

print(f"\nWER (Word Error Rate):")
print(f"  Mean:   {np.mean(wer_scores)*100:.1f}%")
print(f"  Median: {np.median(wer_scores)*100:.1f}%")
print(f"  Std:    {np.std(wer_scores)*100:.1f}%")

print(f"\nCER (Character Error Rate):")
print(f"  Mean:   {np.mean(cer_scores)*100:.1f}%")
print(f"  Median: {np.median(cer_scores)*100:.1f}%")
print(f"  Std:    {np.std(cer_scores)*100:.1f}%")

Duration stats:
  Min:    0.50s
  Max:    29.67s
  Mean:   3.86s
  Median: 2.48s
  Total:  9.3 min

WER (Word Error Rate):
  Mean:   62.0%
  Median: 51.7%
  Std:    47.1%

CER (Character Error Rate):
  Mean:   33.5%
  Median: 20.0%
  Std:    77.7%


In [4]:
# WER distribution breakdown
perfect = sum(1 for w in wer_scores if w == 0)
low = sum(1 for w in wer_scores if 0 < w <= 0.2)
med = sum(1 for w in wer_scores if 0.2 < w <= 0.5)
high = sum(1 for w in wer_scores if 0.5 < w <= 1.0)
terrible = sum(1 for w in wer_scores if w > 1.0)

print("WER Distribution:")
print(f"  Perfect (0%):     {perfect:3d} ({perfect/len(wer_scores)*100:.1f}%)")
print(f"  Low (1-20%):      {low:3d} ({low/len(wer_scores)*100:.1f}%)")
print(f"  Medium (21-50%):  {med:3d} ({med/len(wer_scores)*100:.1f}%)")
print(f"  High (51-100%):   {high:3d} ({high/len(wer_scores)*100:.1f}%)")
print(f"  Terrible (>100%): {terrible:3d} ({terrible/len(wer_scores)*100:.1f}%)")

# WER by dataset
print("\nWER by Dataset:")
from collections import defaultdict
ds_wers = defaultdict(list)
for s, w in zip(samples, wer_scores):
    ds_wers[s["dataset_name"]].append(w)
for ds in sorted(ds_wers.keys()):
    scores = ds_wers[ds]
    print(f"  {ds}: mean={np.mean(scores)*100:.1f}% median={np.median(scores)*100:.1f}% (n={len(scores)})")

WER Distribution:
  Perfect (0%):       5 (3.5%)
  Low (1-20%):        9 (6.2%)
  Medium (21-50%):   58 (40.3%)
  High (51-100%):    66 (45.8%)
  Terrible (>100%):   6 (4.2%)

WER by Dataset:
  MohamedRashad/SADA22: mean=62.0% median=51.7% (n=144)


## 3. Audio Loading Helpers

In [5]:
# Cache for ArrayRecordDataSource objects
_ds_cache = {}

def load_audio(audio_file, record_idx):
    """Load raw audio samples from an ArrayRecord file via gcsfuse."""
    fpath = os.path.join(GCSFUSE_BASE, audio_file)
    if fpath not in _ds_cache:
        _ds_cache[fpath] = grain.ArrayRecordDataSource([fpath])
    ds = _ds_cache[fpath]
    raw_record = ds[record_idx]
    parsed = tf.io.parse_single_example(
        raw_record,
        {
            "audio": tf.io.VarLenFeature(tf.float32),
            "audio_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        },
    )
    audio = tf.sparse.to_dense(parsed["audio"]).numpy()
    audio_len = int(parsed["audio_len"].numpy())
    if audio_len > 0:
        audio = audio[:audio_len]
    return audio.astype(np.float32)


def play_audio(audio, sample_rate=SAMPLE_RATE):
    """Display an IPython audio player for float32 audio samples."""
    return Audio(data=audio, rate=sample_rate)

## 4. Browse Individual Samples

Change `idx` to explore different samples. Listen to the audio and compare the original text with the model's transcription.

In [11]:
def show_sample(idx):
    """Display a single sample with audio player and text comparison."""
    s = samples[idx]
    ref = s["original_text"].strip()
    hyp = s["model_transcription"].strip()
    sample_wer = wer(ref, hyp) * 100 if ref else 0
    sample_cer = cer(ref, hyp) * 100 if ref else 0

    display(Markdown(f"### Sample {idx} / {len(samples)-1}"))
    display(Markdown(f"**Sample ID:** `{s['sample_id']}` &nbsp; **Dataset:** `{s['dataset_name']}`"))
    display(Markdown(f"**Duration:** {s['duration']:.2f}s &nbsp; **WER:** {sample_wer:.1f}% &nbsp; **CER:** {sample_cer:.1f}%"))
    display(Markdown(f"**File:** `{s['audio_file']}` record #{s['record_idx']}"))

    # Load and play audio
    audio = load_audio(s["audio_file"], s["record_idx"])
    display(play_audio(audio))

    # Text comparison (RTL)
    display(HTML(f"""
    <div style="direction: rtl; text-align: right; font-size: 16px; line-height: 2;">
        <p><b>Original (reference):</b><br>
        <span style="color: green; font-size: 18px;">{ref}</span></p>
        <p><b>Model transcription:</b><br>
        <span style="color: blue; font-size: 18px;">{hyp}</span></p>
    </div>
    """))

# Show first sample
show_sample(50)

### Sample 50 / 143

**Sample ID:** `412055b13a7b` &nbsp; **Dataset:** `MohamedRashad/SADA22`

**Duration:** 15.94s &nbsp; **WER:** 41.9% &nbsp; **CER:** 13.4%

**File:** `train/MohamedRashad_SADA22_shard_00000.array_record` record #5343

In [ ]:
# Interactive browser - use the slider to navigate samples
slider = widgets.IntSlider(value=0, min=0, max=len(samples)-1, description="Sample #")
output = widgets.Output()

def on_change(change):
    output.clear_output(wait=True)
    with output:
        show_sample(change["new"])

slider.observe(on_change, names="value")
display(slider, output)
# Trigger initial display
with output:
    show_sample(0)

IntSlider(value=0, description='Sample #', max=143)

Output()

## 5. Worst & Best Transcriptions

Look at the samples with highest and lowest WER to understand where the model struggles.

In [ ]:
# Sort samples by WER
scored = []
for i, s in enumerate(samples):
    ref = s["original_text"].strip()
    hyp = s["model_transcription"].strip()
    if ref:
        w = wer(ref, hyp)
        c = cer(ref, hyp)
        scored.append((i, w, c, s))

scored.sort(key=lambda x: x[1])  # sort by WER ascending

print("=== TOP 10 BEST (lowest WER) ===")
for i, w, c, s in scored[:10]:
    print(f"  [{i:3d}] WER={w*100:5.1f}% CER={c*100:5.1f}% dur={s['duration']:.1f}s ds={s['dataset_name']}")
    print(f"        ref: {s['original_text'][:80]}")
    print(f"        hyp: {s['model_transcription'][:80]}")
    print()

print("\n=== TOP 10 WORST (highest WER) ===")
for i, w, c, s in scored[-10:]:
    print(f"  [{i:3d}] WER={w*100:5.1f}% CER={c*100:5.1f}% dur={s['duration']:.1f}s ds={s['dataset_name']}")
    print(f"        ref: {s['original_text'][:80]}")
    print(f"        hyp: {s['model_transcription'][:80]}")
    print()

In [ ]:
# Listen to a specific worst sample (change the index into scored[-N])
worst_idx = scored[-1][0]  # worst sample
print(f"Listening to worst sample (index {worst_idx}):")
show_sample(worst_idx)

## 6. Try Teacher CoT (Gemini)

This sends the audio + both transcriptions to Gemini to generate a Chain-of-Thought analysis
explaining what's confusing in the audio and why the model made errors.

**Requirements:** Set `GOOGLE_API_KEY` or `GEMINI_API_KEY` environment variable.

```bash
export GOOGLE_API_KEY="your-key-here"
```

In [13]:
COT_PROMPT = """You are an expert Arabic speech analysis assistant. You are given:
1. An audio recording
2. A model-generated transcription: "{model_transcription}"
3. The correct transcription: "{original_text}"

Listen to the audio carefully. Generate a detailed chain-of-thought analysis about:
- Which parts of the audio are unclear or confusing
- Why the model may have made specific errors
- Phonetic similarities that could cause confusion
- Background noise, accent, or pronunciation challenges
- How to correctly identify the unclear segments

Write your analysis in Arabic. Be concise but thorough."""


def audio_to_wav_bytes(audio_samples, sample_rate=SAMPLE_RATE):
    """Convert float32 audio samples to WAV bytes in-memory."""
    audio_int16 = np.clip(audio_samples * 32767, -32768, 32767).astype(np.int16)
    buf = io.BytesIO()
    with wave.open(buf, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio_int16.tobytes())
    return buf.getvalue()


def generate_cot(sample_idx, model_name="gemini-2.5-flash"):
    """Generate CoT analysis for a sample using Gemini."""
    from google import genai
    from google.genai import types

    api_key = os.environ.get("GOOGLE_API_KEY") or os.environ.get("GEMINI_API_KEY")
    if not api_key:
        print("ERROR: Set GOOGLE_API_KEY or GEMINI_API_KEY env var first.")
        print("  In a cell: os.environ['GOOGLE_API_KEY'] = 'your-key'")
        return None

    client = genai.Client(api_key=api_key)
    s = samples[sample_idx]

    # Load audio
    audio = load_audio(s["audio_file"], s["record_idx"])
    wav_bytes = audio_to_wav_bytes(audio)

    # Build prompt
    prompt = COT_PROMPT.format(
        model_transcription=s["model_transcription"],
        original_text=s["original_text"],
    )

    print(f"Calling {model_name} for sample {sample_idx} ({s['sample_id']})...")
    response = client.models.generate_content(
        model=model_name,
        contents=[
            types.Content(
                role="user",
                parts=[
                    types.Part.from_bytes(data=wav_bytes, mime_type="audio/wav"),
                    types.Part.from_text(text=prompt),
                ],
            )
        ],
    )
    return response.text.strip()

In [ ]:
# Pick a sample with errors (change idx as needed)
# Using a sample with medium WER for an interesting CoT
mid_idx = scored[len(scored)//2][0]  # median WER sample
print(f"Selected sample index {mid_idx} (median WER)")
show_sample(mid_idx)

In [ ]:
# Generate CoT for the selected sample
# Uncomment and set your API key first:
# os.environ["GOOGLE_API_KEY"] = "your-key-here"

cot_text = generate_cot(mid_idx)
if cot_text:
    display(Markdown("### Teacher CoT Analysis"))
    display(HTML(f"""
    <div style="direction: rtl; text-align: right; font-size: 15px; line-height: 1.8;
                background: #f5f5f5; padding: 15px; border-radius: 8px; margin: 10px 0;">
        {cot_text}
    </div>
    """))

    # Show what the SFT training target would look like
    s = samples[mid_idx]
    sft_text = f"<think>\n{cot_text}\n</think>\n\n{s['original_text']}"
    display(Markdown("### SFT Training Target (what the model would learn)"))
    display(HTML(f"""
    <div style="direction: rtl; text-align: right; font-size: 14px; line-height: 1.8;
                background: #e8f5e9; padding: 15px; border-radius: 8px; margin: 10px 0;
                font-family: monospace; white-space: pre-wrap;">
        {sft_text}
    </div>
    """))

## 7. Batch CoT for Multiple Samples

Generate CoT for several high-error samples to see patterns.

In [ ]:
import time

# Generate CoT for top 5 worst samples
# Uncomment to run (requires API key)

# cot_results = []
# for rank, (idx, w, c, s) in enumerate(scored[-5:]):
#     print(f"\n{'='*60}")
#     print(f"Rank {rank+1}/5 worst - Sample {idx}, WER={w*100:.1f}%")
#     show_sample(idx)
#     cot = generate_cot(idx)
#     if cot:
#         display(HTML(f"""
#         <div style="direction: rtl; text-align: right; font-size: 14px; line-height: 1.8;
#                     background: #fff3e0; padding: 15px; border-radius: 8px;">
#             <b>CoT:</b><br>{cot}
#         </div>
#         """))
#         cot_results.append({"idx": idx, "wer": w, "cot": cot, **s})
#     time.sleep(6)  # rate limiting (~10 RPM)
#
# print(f"\nGenerated CoT for {len(cot_results)} samples")

## 8. Perfect Transcriptions (Spot Check)

Listen to samples where the model got it exactly right to verify quality.

In [ ]:
# Show first 5 perfect transcriptions
perfect_samples = [(i, w, c, s) for i, w, c, s in scored if w == 0]
print(f"{len(perfect_samples)} samples with perfect WER (0%)\n")

for i, w, c, s in perfect_samples[:5]:
    show_sample(i)
    print("---")